In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
train=pd.read_csv("/content/drive/MyDrive/Minor Final/Dataset/Final_Dataset/Carrot_Local.csv")
train.head()

,Commodity,Date,Average
0,Carrot(Local),6/16/2013,32.5
1,Carrot(Local),6/17/2013,37.5
2,Carrot(Local),6/18/2013,37.5
3,Carrot(Local),6/19/2013,40.0
4,Carrot(Local),6/20/2013,41.0


In [3]:
train['Date']  = train['Date'].apply(pd.to_datetime)

In [4]:
(train["Date"] - train.Date.shift(1))

0         NaT
1      1 days
2      1 days
3      1 days
4      1 days
        ...  
3591   1 days
3592   1 days
3593   1 days
3594   1 days
3595   1 days
Name: Date, Length: 3596, dtype: timedelta64[ns]

In [5]:
(train["Date"] - train.Date.shift(1)).value_counts()

1 days    3470
2 days     104
3 days      14
4 days       4
8 days       1
7 days       1
5 days       1
Name: Date, dtype: int64

In [6]:
train.isna().sum()

Commodity    0
Date         0
Average      0
dtype: int64

In [7]:
px.line(
    data_frame=train,
    x="Date",
    y="Average"
)

In [8]:
resampled_df = train.resample("7D", on ='Date' , ).sum()
px.line(
    data_frame=resampled_df,
#     x="Date",
    y="Average"
)

<ipython-input-8-9834976e19c2>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [9]:
resampled_df

,Average
Date,
2013-06-16,231.00
2013-06-23,162.00
2013-06-30,280.00
2013-07-07,160.00
2013-07-14,265.00
...,...
2023-08-27,860.01
2023-09-03,910.01
2023-09-10,940.01


In [10]:
from sklearn.model_selection import train_test_split

y_train=train['Average']
# Splitting 'y_train' into training and testing sets without shuffling
y_train, y_test = train_test_split(y_train, test_size=0.2, shuffle=False)

In [11]:
# Training size:
print("Train set has {} observations.".format(len(y_train)))
# Test size:
print("Test set has {} observations.".format(len(y_test)))

Train set has 2876 observations.
Test set has 720 observations.


In [12]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
y_train=scaler.fit_transform(y_train.values.reshape(-1,1))
y_test=scaler.transform(y_test.values.reshape(-1,1))

In [13]:
# Sequence Data Preparation
SEQUENCE_SIZE = 10

def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(0 , len(obs) - seq_size):
        window = obs[i:i + seq_size]
        after_window = obs[i + seq_size]
        x.append(window)
        y.append(after_window)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Using y_test only
x_train, y_train = to_sequences(SEQUENCE_SIZE, y_train)
x_test, y_test = to_sequences(SEQUENCE_SIZE, y_test)

# Setup data loaders for batch
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

<ipython-input-13-08401c7e56ea>:12: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)



In [14]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [15]:
try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: using Google CoLab
Using device: cpu


In [16]:
# Model definition using Transformer
class TransformerModel(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(TransformerModel, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model )
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers,enable_nested_tensor=True)
        self.decoder = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.decoder(x[:, -1, :])
        return x

model = TransformerModel(
    input_dim = 1,
    d_model = 32,
    nhead = 16,
    num_layers = 2,
    dropout= 0.05,
).to(device)

In [17]:
from torch.optim.lr_scheduler import ConstantLR
# Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)
scheduler = ConstantLR(optimizer)
epochs = 100
early_stop_count = 0
min_val_loss = float('inf')
train_losses = []
validation_losses = []
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step()
    validation_losses.append(val_loss)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    # if early_stop_count >= 5:
    #     print("Early stopping!")
    #     break
    print(f"Epoch {epoch + 1}/{epochs},Train Loss {loss.item()} Validation Loss: {val_loss:.4f}")

Epoch 1/100,Train Loss 0.31869208812713623 Validation Loss: 0.1180
Epoch 2/100,Train Loss 0.37760376930236816 Validation Loss: 0.1213
Epoch 3/100,Train Loss 0.3656862676143646 Validation Loss: 0.1160
Epoch 4/100,Train Loss 0.3792599141597748 Validation Loss: 0.1260
Epoch 5/100,Train Loss 0.38013795018196106 Validation Loss: 0.1221
Epoch 6/100,Train Loss 0.44829440116882324 Validation Loss: 0.1426
Epoch 7/100,Train Loss 0.5684502124786377 Validation Loss: 0.2309
Epoch 8/100,Train Loss 0.47963201999664307 Validation Loss: 0.1900
Epoch 9/100,Train Loss 0.43658915162086487 Validation Loss: 0.1563
Epoch 10/100,Train Loss 0.4194728434085846 Validation Loss: 0.1225
Epoch 11/100,Train Loss 0.40674614906311035 Validation Loss: 0.1361
Epoch 12/100,Train Loss 0.44883882999420166 Validation Loss: 0.1485
Epoch 13/100,Train Loss 0.48335936665534973 Validation Loss: 0.1357
Epoch 14/100,Train Loss 0.38496580719947815 Validation Loss: 0.1192
Epoch 15/100,Train Loss 0.38436591625213623 Validation Loss: 

In [18]:
torch.save(model,"/content/drive/MyDrive/Minor Final/Transformer_Final model/Carrot.pth")

In [ ]:
model = torch.load("/content/drive/MyDrive/Minor Final/Transformer_Final model/Carrot.pth", map_location=torch.device('cpu'))

In [ ]:
criterion = nn.MSELoss()
model.eval()
predictions = []
valid_loss = []
with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch)
        predictions.extend(outputs.tolist())
        validation_loss = criterion(outputs, y_batch)
        valid_loss.append(validation_loss.item())

# Convert predictions to a 2D numpy array
predictions = np.array(predictions).reshape(-1, 1)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)

# Calculate R^2 score
r2 = r2_score(y_test, predictions)

# Print scores
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

RMSE: 0.3263
MAE: 0.2084
MSE: 0.1065
R^2 Score: 0.9264
